## Assignment week 3 - Neighbourhoods of Toronto


### Aim
exploring, segmenting and clustering the neighbourhoods of Toronto by using data from Wikipedia and FourSquare


### 1. Get  and clean the data about Toronto neighbourhoods from Wikipedia



In [2]:
# first install wikipedia-api
!conda install -c conda-forge wikipedia --yes 

Solving environment: done

## Package Plan ##

  environment location: /Users/Elisabeth/anaconda3

  added / updated specs:
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wikipedia-1.4.0            |             py_2          13 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          13 KB

The following NEW packages will be INSTALLED:

  wikipedia          conda-forge/noarch::wikipedia-1.4.0-py_2



wikipedia-1.4.0      | 13 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
### Import libraries needed for whole project ###

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wikipedia

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print('Libraries imported.')

Libraries imported.


#### 1.1. Load data from wikipedia

In [88]:
# Get the html source
html = wikipedia.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")

# save it as pd.dataframe 
df = pd.read_html(html)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 1.2 Clean data

In [177]:
# exlcude rows with borough not assigned
df2=df[df.Borough!= 'Not assigned']
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [195]:
# number of valid Toronto postcodes
postcode=df2.Postcode.unique()
len(postcode)

103

In [133]:
# find not assigned neighbourhoods
df2[df2.Neighbourhood=='Not assigned']

,Postcode,Borough,Neighbourhood
7,M7A,Queen's Park,Not assigned


In [172]:
# assign Borough name to not assigned neighbourhoods
df2.loc[(df2.Neighbourhood == 'Not assigned'),'Neighbourhood']= df2['Borough']
df2


/Users/Elisabeth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [173]:
# combine neighborhoods with same postal code
df_neighbourhoods=df2.groupby("Postcode")['Neighbourhood'].apply(lambda Neighbourhood: ','.join(Neighbourhood))

df_neighbourhoods

Postcode
M1B                                        Rouge,Malvern
M1C                 Highland Creek,Rouge Hill,Port Union
M1E                      Guildwood,Morningside,West Hill
M1G                                               Woburn
M1H                                            Cedarbrae
M1J                                  Scarborough Village
M1K            East Birchmount Park,Ionview,Kennedy Park
M1L                        Clairlea,Golden Mile,Oakridge
M1M        Cliffcrest,Cliffside,Scarborough Village West
M1N                           Birch Cliff,Cliffside West
M1P    Dorset Park,Scarborough Town Centre,Wexford He...
M1R                                     Maryvale,Wexford
M1S                                            Agincourt
M1T                Clarks Corners,Sullivan,Tam O'Shanter
M1V    Agincourt North,L'Amoreaux East,Milliken,Steel...
M1W                                      L'Amoreaux West
M1X                                          Upper Rouge
M2H                   

In [174]:
# make dataframe with unique postcode and borough
df_borough=df2[['Postcode','Borough']].drop_duplicates()

In [188]:
# merge both dataframes on Postcode
df_final=df_borough.join(df_neighbourhoods,'Postcode')
df_final.reset_index(drop=True,inplace=True)
df_final.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [184]:
# check if data of all postcodes is included use .shape function
df_final.shape

(103, 3)

### 2. Getting lattitude and longitude data

(all needed libraries where imported above as part of 1.)

### 3. Cluster analysis of Toronto Neighborhoods